### AGNQSO Summary catalogue generator

This notebook demonstrates the catalog generation.

All functions and final wrapper script to live in py/ * .py.

If you are on NERSC please select 'DESI main' as your kernel.

Notebook direct contirbutions:

* Alexander, D (Univ. of Durham, Durham, UK) (VI merging done by DA)
* Alfarsy, R (Univ. of Portsmouth, Portsmouth, UK)
* Canning, B (Univ. of Portsmouth, Portsmouth, UK)
* Chaussidon, E (CEA Saclay, Paris, France) (QSO catalogs generated by EC et al.)
* Juneau, S (NOIRLab, Arizona, USA)
* Mezcua, M (Institut de Ciències de l'Espai, Barcelona, Spain)
* Moustakas, J (Siena College, New York, USA) (FastSpecFit catalogues by JM)
* Pucha, R (Univ. of Arizona, Arizona, USA) 

## Docs:
Readme:
Wiki: 
Github: 

Directions for VACs: 
- EDR: https://desi.lbl.gov/trac/wiki/Pipeline/Releases/EDR/Planning/ValueAdded
- DR1: https://desi.lbl.gov/trac/wiki/Pipeline/Releases/DR1/Planning/ValueAdded

Our VAC directory: /global/cfs/cdirs/desi/science/gqp/agncatalog/  

## Imports

In [ ]:
# General imports
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os.path
import gc
import glob

# Import Astropy libraries - useful for many astronomy related function
from astropy.table import Table, join, hstack
from astropy.io import fits
# Fast FITS file I/O access
import fitsio

# DESI modules
#from desispec.zcatalog import find_primary_spectra, create_summary_catalog  # at NERSC needs DESI master
#from desitarget.targetmask import desi_mask, bgs_mask, scnd_mask      # For the main survey
#from desiutil.bitmask import BitMask
#from desiutil.annotate import annotate_table, annotate_fits

# GQP_CODE
import sys
sys.path.append("../py/")
import set_agn_masksDESI
# import AgnCats.py.set_agn_masksDESI

#https://www.legacysurvey.org/viewer?ra=10.1572&dec=-0.3316&layer=ls-dr9&zoom=16

# Workflow

Deveopment code in: /global/homes/b/bcanning/AGNQSO_summary_catalog/

1. Read Edmond threshold catalog
   * OBJ_TYPE = TGT
   * low-z star cut   
   
   * Note: Fuji has defaut 0.95 c_thresh for all targets
     - If BGS target (not QSO, not ELG, not LRG): RR SPECTYPE=QSO and (QN C_LINE_BEST>0.6 or MgII)
     - If ELG (not QSO): RR SPECTYPE=QSO and QN C_LINE_BEST>0.6
     - If QSO threshold 'QN_C_LINE_BEST' > 0.95 (Default)

2. Generate (or read) summary catalog
   * Join to add 'TSNR2_LRG','SV_NSPEC','SV_PRIMARY','ZCAT_NSPEC','ZCAT_PRIMARY'
   * Generate 'QN_C_LINE_BEST' and 'QN_C_LINE_SECOND_BEST'

3. Join FastSpecFit v2 - specific columns

4. Read yaml file

5. Set QSO_MASKBITS part of AGN_MASKBITS

6. Set BPT bits 
   * Update AGN_MASKBITS 

7. Join multiwave survey

8. Write catalog

## Basic Info (edit for choosing data release)

In [ ]:
GQP_AGNcat_dir='/global/cfs/cdirs/desi/science/gqp/agncatalog/'

# Which spectroscopic release
specprod = 'fuji'
#specprod = 'guadalupe'
#specprod = 'iron'
#specprod = 'loa'

## EDR version
if specprod=='fuji':
    
    #### QSO-maker file
    # Edmonds catalogue from QSO maker keeping all columns
    path_qsom = f'/global/cfs/cdirs/desi/users/edmondc/QSO_catalog/{specprod}/'  #NERSC
    file_qsom = path_qsom+f'QSO_cat_{specprod}_healpix_all_targets_v2.fits'
    
    # FastSpecFit file
    fast_dir = f'/global/cfs/cdirs/desi/spectro/fastspecfit/{specprod}/v3.2/catalogs/'
    fastspec_file = fast_dir+f'fastspec-{specprod}.fits'
    #fastphot_file = fast_dir+f'fastphot-{specprod}.fits'
    
    # Redshift catalog
    #file_zpix_sum_cat=dir_for_tmp+'zpix-'+specprod+'-summary.fits'
    # Using the public EDR version of the zcat VAC
    file_zpix_sum_cat = '/global/cfs/cdirs/desi/public/edr/vac/edr/zcat/fuji/v1.0/zall-pix-edr-vac.fits'

## DR1 version
if specprod=='iron':

    ## QSO-maker path (NOTE: from merge_QSOmaker.ipynb)
    path_qsom = f'/global/cfs/cdirs/desi/science/gqp/agncatalog/qsomaker/iron/'

    # Needed to make DR1 version after asking Edmond to run on all targets / all surveys
    file_qsom = path_qsom+'QSO_cat_iron_healpix_all_targets_v1.fits'

    # FastSpecFit file
    fast_dir = f'/global/cfs/cdirs/desi/spectro/fastspecfit/{specprod}/v2.1/catalogs/'
    # File with all DR1
    fastspec_file = fast_dir+f'fastspec-{specprod}.fits'

    # zcat
    file_zpix_sum_cat = '/global/cfs/cdirs/desi/spectro/redux/iron/zcatalog/v1/zall-pix-iron.fits'

## DR2 version
if specprod=='loa':

    # FastSpecFit file
    fast_dir = f'/global/cfs/cdirs/desi/vac/dr2/fastspecfit/loa/v1.0/catalogs/'

    files = glob.glob(fast_dir+'/fast*')
    filenames = [s.split('/')[-1] for s in files]
    tmp = [((s.replace('-','_')).replace('fastspec', 'QSO_cat')).replace('.fits','') for s in filenames]  
    qsonames = [s+'_healpix_all_targets_v1.fits' for s in tmp]
    zcatnames = [s.replace('fastspec-loa','zpix') for s in filenames]  

    file_i = 0
    
    #print(np.c_[filenames, qsonames, zcatnames])
    
    # File with all DR1
    fastspec_file = fast_dir+filenames[file_i]

    ## QSO-maker path (NOTE: from merge_QSOmaker.ipynb)
    path_qsom = f'/global/cfs/cdirs/desi/science/gqp/agncatalog/qsomaker/loa/'
    file_qsom = path_qsom+qsonames[file_i]

    # zcat
    path_zcat = f'/global/cfs/cdirs/desi/spectro/redux/loa/zcatalog/v1/'
    file_zpix_sum_cat = path_zcat+zcatnames[file_i]

    print(fastspec_file)
    print(file_qsom)
    print(file_zpix_sum_cat)
    
## Put catalog in this temporary location then copy over to correct DR and 
## version number when happy with a new version
dir_for_cat=GQP_AGNcat_dir+'catalog/'
dir_for_tmp=GQP_AGNcat_dir+'tmp/'

## Choice to use healpix-based catalogs
filetype = 'healpix'
specgroup_type = 'zpix'

In [ ]:
# Main identifiers for Joins
keys_for_join=['TARGETID','SURVEY','PROGRAM']

In [ ]:
# Data model
#
# Question about whether to keep: 
#  - LS_ID - this one from FSF
#  - other Tractor cols (MORPHTYPE, MASKBITS, PHOTSYS)
#  - EBV_1 (where does this one come from? from joining two tables with EBV??)
#  - FIBERFLUX* and FIBERTOTFLUX*
#  + replaced HPXPIXEL with HEALPIX (updated name); added BGS targeting cols to find BGS_WISE
#  + mjd information min, mean, max, zcat and primary stuff: https://github.com/desihub/desispec/blob/master/py/desispec/zcatalog.py
#  + removed QSO_MASKBITS as repeated in agn_maskbits
#  + where does the Z come from - FSF or QSO maker - make it FSF!
#
#final_cols=['TARGETID','SURVEY','PROGRAM','HEALPIX','Z','ZERR','ZWARN','SPECTYPE','COADD_FIBERSTATUS','TARGET_RA','TARGET_DEC',\
#            'MORPHTYPE','EBV_1','MASKBITS',\
#            'DESI_TARGET','SCND_TARGET','BGS_TARGET','COADD_NUMEXP','COADD_EXPTIME','CMX_TARGET',\
#            'SV1_DESI_TARGET','SV2_DESI_TARGET','SV3_DESI_TARGET','SV1_BGS_TARGET','SV2_BGS_TARGET','SV3_BGS_TARGET',\
#            'SV1_SCND_TARGET','SV2_SCND_TARGET','SV3_SCND_TARGET',\
#            'TSNR2_LYA','TSNR2_QSO','TSNR2_LRG',\
#            'DELTA_CHI2_MGII','A_MGII','SIGMA_MGII','B_MGII','VAR_A_MGII','VAR_SIGMA_MGII','VAR_B_MGII',\
#            'Z_RR','Z_QN','C_LYA','C_CIV','C_CIII','C_MgII','C_Hbeta','C_Halpha','Z_LYA','Z_CIV','Z_CIII','Z_MgII','Z_Hbeta','Z_Halpha',\
#            'SV_NSPEC','SV_PRIMARY','ZCAT_NSPEC','ZCAT_PRIMARY',\
#            'QN_C_LINE_BEST','QN_C_LINE_SECOND_BEST','QSO_MASKBITS','agn_maskbits','AGN_TYPE',\
#            'PHOTSYS','LS_ID','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z','FIBERTOTFLUX_G','FIBERTOTFLUX_R','FIBERTOTFLUX_Z'
#.           'MIN_MJD','MEAN_MJD','MAX_MJD']

## SJ: should we keep QN_C_LINE_BEST?
## SJ: Do we need 'MEAN_MJD'?

## EDR version
if specprod=='fuji': 
    final_cols=['TARGETID', 'SURVEY', 'PROGRAM', 'HEALPIX', 'Z', 'ZERR', 'ZWARN', 'SPECTYPE',
                'AGN_MASKBITS', 'OPT_UV_TYPE', 'IR_TYPE', 'COADD_FIBERSTATUS',
                'TARGET_RA', 'TARGET_DEC', 'LS_ID', 'MIN_MJD','MEAN_MJD','MAX_MJD','COADD_NUMEXP', 'COADD_EXPTIME',
                'SV_PRIMARY','ZCAT_PRIMARY','DESI_TARGET', 'SCND_TARGET', 'BGS_TARGET', 'CMX_TARGET',
                'SV1_DESI_TARGET', 'SV2_DESI_TARGET', 'SV3_DESI_TARGET', 'SV1_BGS_TARGET', 'SV2_BGS_TARGET', 'SV3_BGS_TARGET',
                'SV1_SCND_TARGET', 'SV2_SCND_TARGET', 'SV3_SCND_TARGET']

## DR1 version
if specprod=='iron': 
    final_cols=['TARGETID', 'SURVEY', 'PROGRAM', 'HEALPIX', 'Z', 'ZERR', 'ZWARN', 'SPECTYPE',
                'AGN_MASKBITS', 'OPT_UV_TYPE', 'IR_TYPE', 'COADD_FIBERSTATUS',
                'TARGET_RA', 'TARGET_DEC', 'LS_ID', 'MIN_MJD','MEAN_MJD','MAX_MJD','COADD_NUMEXP', 'COADD_EXPTIME',
                'SV_PRIMARY','MAIN_PRIMARY','ZCAT_PRIMARY','DESI_TARGET', 'SCND_TARGET', 'BGS_TARGET', 'CMX_TARGET',
                'SV1_DESI_TARGET', 'SV2_DESI_TARGET', 'SV3_DESI_TARGET', 'SV1_BGS_TARGET', 'SV2_BGS_TARGET', 'SV3_BGS_TARGET',
                'SV1_SCND_TARGET', 'SV2_SCND_TARGET', 'SV3_SCND_TARGET']

if specprod=='loa': 
    final_cols=['TARGETID', 'SURVEY', 'PROGRAM', 'HEALPIX', 'Z', 'ZERR', 'ZWARN', 'SPECTYPE',
                'AGN_MASKBITS', 'OPT_UV_TYPE', 'IR_TYPE', 'COADD_FIBERSTATUS',
                'TARGET_RA', 'TARGET_DEC', 'LS_ID', 'MIN_MJD','MEAN_MJD','MAX_MJD','COADD_NUMEXP', 'COADD_EXPTIME',
                'SV_PRIMARY','MAIN_PRIMARY','ZCAT_PRIMARY','DESI_TARGET', 'SCND_TARGET', 'BGS_TARGET', 'CMX_TARGET',
                'SV1_DESI_TARGET', 'SV2_DESI_TARGET', 'SV3_DESI_TARGET', 'SV1_BGS_TARGET', 'SV2_BGS_TARGET', 'SV3_BGS_TARGET',
                'SV1_SCND_TARGET', 'SV2_SCND_TARGET', 'SV3_SCND_TARGET']


## Column for "convenience" extension with values for plotting
ext2_cols=['TARGETID','SURVEY','PROGRAM','LOGMSTAR',
           'FLUX_W1','FLUX_W2','FLUX_W3',
           'FLUX_IVAR_W1','FLUX_IVAR_W2','FLUX_IVAR_W3',
           'CIV_1549_FLUX','CIV_1549_FLUX_IVAR', 'CIV_1549_SIGMA',
           'MGII_2796_FLUX','MGII_2796_FLUX_IVAR','MGII_2796_SIGMA',
           'MGII_2803_FLUX','MGII_2803_FLUX_IVAR', 'MGII_2803_SIGMA',
           'OII_3726_FLUX','OII_3726_FLUX_IVAR','OII_3726_EW','OII_3726_EW_IVAR',
           'OII_3729_FLUX','OII_3729_FLUX_IVAR','OII_3729_EW','OII_3729_EW_IVAR',
           'NEV_3426_FLUX','NEV_3426_FLUX_IVAR',
           'HEII_4686_FLUX','HEII_4686_FLUX_IVAR',
           'HBETA_EW','HBETA_EW_IVAR','HBETA_FLUX','HBETA_FLUX_IVAR',
           'HBETA_BROAD_FLUX', 'HBETA_BROAD_FLUX_IVAR', 'HBETA_BROAD_SIGMA','HBETA_BROAD_CHI2',
           'OIII_5007_FLUX','OIII_5007_FLUX_IVAR','OIII_5007_SIGMA',
           'OI_6300_FLUX','OI_6300_FLUX_IVAR',
           'HALPHA_EW', 'HALPHA_EW_IVAR', 'HALPHA_FLUX','HALPHA_FLUX_IVAR',
           'HALPHA_BROAD_FLUX','HALPHA_BROAD_FLUX_IVAR','HALPHA_BROAD_VSHIFT','HALPHA_BROAD_SIGMA',
           'NII_6584_FLUX','NII_6584_FLUX_IVAR',
           'SII_6716_FLUX','SII_6716_FLUX_IVAR',
           'SII_6731_FLUX','SII_6731_FLUX_IVAR']

# SJ: removed: 'HALPHA_SIGMA'
#     added: 'HALPHA_EW_IVAR'

# Longer list (all that we use?)

# ext2_cols=['TARGETID','SURVEY','PROGRAM','LOGMSTAR',\
#            'CIV_1549_FLUX','CIV_1549_FLUX_IVAR', 'CIV_1549_SIGMA',\
#            'MGII_2796_FLUX','MGII_2796_FLUX_IVAR','MGII_2796_SIGMA',\
#            'MGII_2803_FLUX','MGII_2803_FLUX_IVAR', 'MGII_2803_SIGMA',\
#            'OII_3726_FLUX','OII_3726_FLUX_IVAR','OII_3726_EW',\
#            'NEV_3426_FLUX','NEV_3426_FLUX_IVAR',\
#            'HEII_4686_FLUX','HEII_4686_FLUX_IVAR',\
#            'HBETA_EW','HBETA_FLUX','HBETA_FLUX_IVAR',\
#            'HBETA_BROAD_FLUX', 'HBETA_BROAD_FLUX_IVAR', 'HBETA_BROAD_VSHIFT','HBETA_BROAD_SIGMA','HBETA_BROAD_CHI2',\
#            'OIII_5007_FLUX','OIII_5007_FLUX_IVAR','OIII_5007_SIGMA',\
#            'OI_6300_FLUX','OI_6300_FLUX_IVAR',\
#            'HALPHA_EW', 'HALPHA_FLUX','HALPHA_FLUX_IVAR','HALPHA_SIGMA', \
#            'HALPHA_BROAD_FLUX','HALPHA_BROAD_FLUX_IVAR','HALPHA_BROAD_VSHIFT','HALPHA_BROAD_SIGMA',\
#            'NII_6584_FLUX','NII_6584_FLUX_IVAR',\
#            'SII_6716_FLUX','SII_6716_FLUX_IVAR',\
#            'SII_6731_FLUX','SII_6731_FLUX_IVAR']

print(final_cols)

# Define cuts that might be wanted

The below function should be run on the final joined catalogs as not all keywords exist otherwise.

We are not making any cuts currnetly. 

In [ ]:
# def cut_fiberstatus(input_table):
#     ''' 
#     keep only objects with 'COADD_FIBERSTATUS' == 0
#     '''
#     keep = (input_table['COADD_FIBERSTATUS']==0)
#     return input_table[keep]

# def cut_npixels(input_table):
#     ''' 
#     keep only objects with 'NPIXELS' > 0 (signifying they have a coadded spectrum)
#     '''
#     keep = (input_table['NPIXELS']>0)
#     return input_table[keep]

# def cut_zwarn(input_table):
#     ''' 
#     keep only objects with                  (zb['ZWARN'] & ZWarningMask.NODATA == 0))[0]
# https://fastspecfit.readthedocs.io/en/latest/vacs.html#sample-selection
#     '''
#     keep = (input_table['ZWARN'] & ZWarningMask.NODATA ==0)  # might not be written correctly...
#     return input_table[keep]

def cut_objtype(T):
    ''' 
    keep only objects with 'OBJTYPE' == 'TGT'
    '''
    keep = (T['OBJTYPE']=='TGT')
    return T[keep]

def cut_lowz_star(T):
    ''' 
    keep only objects with redshift greater than 0.001
    '''
    keep = (T['Z']>0.001)
    return T[keep]

def cut_exptime(T):
    ''' 
    keep only objects with COADD_EXPTIME greater than 0.0
    '''
    keep = (T['COADD_EXPTIME']>0.0)
    return T[keep]

# def cut_lowz_galfragments(input_table):
#     ''' 
#     keep only objects with 
#     '''
#     keep = (input_table['']==)
#     return input_table[keep]

### cuts added by EC but leaving in here for discussion with SJ

# #### Notes/Questions
# - How to treat objects that might have more than one target type?
# - Correct bump at z~3.7:
# ```
#     sel_pb_redshift = (QSO_cat['Z'] > 3.65) & ((QSO_cat['C_LYA']<0.95) | (QSO_cat['C_CIV']<0.95))
# ```

In [ ]:
#test1=cut_objtype(T_qsom)
#test2=cut_lowz_star(T_qsom)
#print(len(test2))

## 1. QSO-maker Cat

In [ ]:
## SJ: will exclude the targeting cols because we'll add them from the zcat VAC instead 
#qsom_cols=['TARGETID','Z','ZERR','ZWARN','SPECTYPE','COADD_FIBERSTATUS','TARGET_RA','TARGET_DEC','MORPHTYPE','EBV','MASKBITS','DESI_TARGET','SCND_TARGET','COADD_NUMEXP','COADD_EXPTIME','CMX_TARGET','SV1_DESI_TARGET','SV2_DESI_TARGET','SV3_DESI_TARGET','SV1_SCND_TARGET','SV2_SCND_TARGET','SV3_SCND_TARGET','TSNR2_LYA','TSNR2_QSO','DELTA_CHI2_MGII','A_MGII','SIGMA_MGII','B_MGII','VAR_A_MGII','VAR_SIGMA_MGII','VAR_B_MGII','Z_RR','Z_QN','C_LYA','C_CIV','C_CIII','C_MgII','C_Hbeta','C_Halpha','Z_LYA','Z_CIV','Z_CIII','Z_MgII','Z_Hbeta','Z_Halpha','QSO_MASKBITS','HPXPIXEL','SURVEY','PROGRAM']

# Current choice for Iron
qsom_cols=['TARGETID','Z','ZERR','ZWARN','SPECTYPE','COADD_FIBERSTATUS','TARGET_RA','TARGET_DEC',
           'MORPHTYPE','MASKBITS','COADD_NUMEXP','COADD_EXPTIME','TSNR2_LYA','TSNR2_QSO',
           'Z_RR','Z_QN','C_LYA','C_CIV','C_CIII','C_MgII','C_Hbeta','C_Halpha',
           'QSO_MASKBITS','SURVEY','PROGRAM']
# Trying without these (they're not used):  
#   'DELTA_CHI2_MGII','A_MGII','SIGMA_MGII','B_MGII','VAR_A_MGII','VAR_SIGMA_MGII','VAR_B_MGII',\
#   'Z_LYA','Z_CIV','Z_CIII','Z_MgII','Z_Hbeta','Z_Halpha'

print(qsom_cols)

In [ ]:
%%time
# Takes about 25sec for dr1
T_qsom = Table(fitsio.read(file_qsom, columns=qsom_cols, ext=1)) 
# For one npix in dr2 takes 30.5 s

In [ ]:
print(len(T_qsom))

## Remove stars with a low redshift cut
T_qsom = cut_lowz_star(T_qsom)

print(len(T_qsom))

# 18260646
# 18165695

In [ ]:
print(len(T_qsom[T_qsom['COADD_EXPTIME']==0.]))

## Remove missing spectra that have COADD_EXPTIME=0.0
T_qsom = cut_exptime(T_qsom)
print(len(T_qsom))

## 2. Use the Redshift Summary (zcat) VAC

The Redshift Summary Catalog VAC supersedes the original redshift catalogs for Fuji (EDR) as described [here](https://data.desi.lbl.gov/doc/releases/edr/vac/zcat/).

In [ ]:
# from https://github.com/desihub/desispec/blob/master/py/desispec/zcatalog.py
# for fugi 2Gb file (approx 5 mins)
if os.path.isfile(file_zpix_sum_cat):
   print('zpix summary file exists - using existing copy')
else:
   print('ERROR: zpix summary file NOT FOUND')
#   print('Creating zpix summary file - this is a couple of Gb for fuji and may take 5 mins')
#   create_summary_catalog(specprod, specgroup = specgroup_type, all_columns = True, columns_list = None, output_filename = file_zpix_sum_cat)

In [ ]:
# from desispec.io.table import read_table

In [ ]:
%%time
if specprod == 'fuji':
    need_cols = ['TARGETID','SURVEY','PROGRAM','HEALPIX','TSNR2_LRG','SV_NSPEC','SV_PRIMARY',
                 'ZCAT_NSPEC','ZCAT_PRIMARY','MIN_MJD','MEAN_MJD','MAX_MJD', 'OBJTYPE'] # fuji
if specprod == 'guadalupe':
    need_cols = ['TARGETID','SURVEY','PROGRAM','HEALPIX','TSNR2_LRG','ZCAT_NSPEC','ZCAT_PRIMARY',
                 'MIN_MJD','MEAN_MJD','MAX_MJD','OBJTYPE'] # guadalupe
    # for guadalupe, iron, etc.: also add MAIN_PRIMARY and MAIN_NSPEC
if specprod == 'iron':
    need_cols = ['TARGETID','SURVEY','PROGRAM','HEALPIX','TSNR2_LRG','ZCAT_NSPEC','ZCAT_PRIMARY',
                 'SV_NSPEC','SV_PRIMARY','MAIN_PRIMARY','MAIN_NSPEC','MIN_MJD','MEAN_MJD','MAX_MJD','OBJTYPE']    

if specprod == 'loa':
    need_cols = ['TARGETID','SURVEY','PROGRAM','HEALPIX','TSNR2_LRG','ZCAT_NSPEC','ZCAT_PRIMARY',
                 'SV_NSPEC','SV_PRIMARY','MAIN_PRIMARY','MAIN_NSPEC','MIN_MJD','MEAN_MJD','MAX_MJD','OBJTYPE']    

# Replace targeting columns with updated version from zcat VAC (for Fuji), keeping BGS to find BGS_WISE targets
target_cols = ['DESI_TARGET','BGS_TARGET','SCND_TARGET','CMX_TARGET','SV1_DESI_TARGET','SV1_BGS_TARGET','SV1_SCND_TARGET',
               'SV2_DESI_TARGET','SV2_BGS_TARGET','SV2_SCND_TARGET','SV3_DESI_TARGET','SV3_BGS_TARGET','SV3_SCND_TARGET']

## SJ: for faster performance, only read the desired columns with fitsio() from the zpix_sum file
T_zpixsum_cut = Table(fitsio.read(file_zpix_sum_cat, columns=need_cols+target_cols, ext=1))
#T_zpixsum_cut = Table(fitsio.read(file_zpix_sum_cat, ext=1))

In [ ]:
T_zpixsum_cut.columns

In [ ]:
%%time
## This is slow! Would it be faster to work with Pandas DataFrames?
## Takes >2min for Iron/DR1 without indexing tables
T_qsom_zpixsum = join(T_qsom, T_zpixsum_cut, keys=keys_for_join, join_type='left')

In [ ]:
# making sure this is the same size as before
print(len(T_qsom_zpixsum))
if len(T_qsom) == len(T_qsom_zpixsum):
    print('Same length - all good')
else:
    print('The joined QSO maker and summary catalog df is not the same size as the QSO maker catalog')
    print('Something went wrong!')

In [ ]:
# free memory
del T_qsom
del T_zpixsum_cut
gc.collect()

In [ ]:
## Already applied for Iron/DR1 but will run anyway for now
T_qsom_zpixsum = cut_objtype(T_qsom_zpixsum)
print(len(T_qsom_zpixsum))

In [ ]:
## SJ: Where is this used??
# it's now added to QSO-maker for Iron (DR1) so could read it and save it

## Adding two columns we need for the cuts
#a = np.array([T_qsom_zpixsum['C_LYA'], T_qsom_zpixsum['C_CIV'], T_qsom_zpixsum['C_CIII'], \
#              T_qsom_zpixsum['C_MgII'], T_qsom_zpixsum['C_Hbeta'], T_qsom_zpixsum['C_Halpha']])
#T_qsom_zpixsum['QN_C_LINE_BEST'] = [max(l) for l in (a.input_table).tolist()]
#T_qsom_zpixsum['QN_C_LINE_SECOND_BEST'] = [sorted(l)[-2] for l in (a.input_table).tolist()]

## 3. Join FastSpecFit

Original list but let's remove anything unnecessary:
```
fsf_data_cols=['TARGETID','SURVEY','PROGRAM','LOGMSTAR',\
               'CIV_1549_FLUX','CIV_1549_FLUX_IVAR', 'CIV_1549_VSHIFT','CIV_1549_SIGMA',\
               'MGII_2796_FLUX','MGII_2796_FLUX_IVAR', 'MGII_2796_VSHIFT','MGII_2796_SIGMA',\
               'MGII_2803_FLUX','MGII_2803_FLUX_IVAR', 'MGII_2803_SIGMA',\
               'OII_3726_FLUX','OII_3726_FLUX_IVAR','OII_3726_EW','OII_3726_EW_IVAR',\
               'OII_3729_FLUX','OII_3729_FLUX_IVAR','OII_3729_EW','OII_3729_EW_IVAR',\
               'NEV_3426_FLUX','NEV_3426_FLUX_IVAR','NEV_3426_VSHIFT','NEV_3426_SIGMA',\
               'HEII_4686_FLUX','HEII_4686_FLUX_IVAR',\
               'HBETA_EW','HBETA_EW_IVAR','HBETA_FLUX','HBETA_FLUX_IVAR','HBETA_VSHIFT','HBETA_SIGMA',\
               'HBETA_BROAD_FLUX', 'HBETA_BROAD_FLUX_IVAR', 'HBETA_BROAD_VSHIFT','HBETA_BROAD_SIGMA','HBETA_BROAD_CHI2',\
               'OIII_5007_FLUX','OIII_5007_FLUX_IVAR','OIII_5007_VSHIFT','OIII_5007_SIGMA',\
               'OI_6300_FLUX','OI_6300_FLUX_IVAR','OI_6300_VSHIFT','OI_6300_SIGMA',\
               'HALPHA_EW', 'HALPHA_FLUX','HALPHA_FLUX_IVAR','HALPHA_VSHIFT','HALPHA_SIGMA', \
               'HALPHA_BROAD_FLUX','HALPHA_BROAD_FLUX_IVAR','HALPHA_BROAD_VSHIFT','HALPHA_BROAD_SIGMA',\
               'NII_6584_FLUX','NII_6584_FLUX_IVAR','NII_6584_VSHIFT','NII_6584_SIGMA',\
               'SII_6716_FLUX','SII_6716_FLUX_IVAR','SII_6716_VSHIFT','SII_6716_SIGMA',\
               'SII_6731_FLUX','SII_6731_FLUX_IVAR','SII_6731_VSHIFT','SII_6731_SIGMA']
```

In [ ]:
## SJ: downselected closer to the list for Extension 2 (e.g., removed the VSHIFT and unnecessary cols)
fsf_data_cols=['TARGETID','SURVEY','PROGRAM','LOGMSTAR',
               'CIV_1549_FLUX','CIV_1549_FLUX_IVAR', 'CIV_1549_SIGMA',
               'MGII_2796_FLUX','MGII_2796_FLUX_IVAR', 'MGII_2796_SIGMA',
               'MGII_2803_FLUX','MGII_2803_FLUX_IVAR', 'MGII_2803_SIGMA',
               'OII_3726_FLUX','OII_3726_FLUX_IVAR','OII_3726_EW','OII_3726_EW_IVAR',
               'OII_3729_FLUX','OII_3729_FLUX_IVAR','OII_3729_EW','OII_3729_EW_IVAR',
               'NEV_3426_FLUX','NEV_3426_FLUX_IVAR',
               'HEII_4686_FLUX','HEII_4686_FLUX_IVAR',
               'HBETA_EW','HBETA_EW_IVAR','HBETA_FLUX','HBETA_FLUX_IVAR',
               'HBETA_BROAD_FLUX', 'HBETA_BROAD_FLUX_IVAR', 'HBETA_BROAD_SIGMA','HBETA_BROAD_CHI2',
               'OIII_5007_FLUX','OIII_5007_FLUX_IVAR','OIII_5007_SIGMA',
               'OI_6300_FLUX','OI_6300_FLUX_IVAR',
               'HALPHA_EW','HALPHA_EW_IVAR', 'HALPHA_FLUX','HALPHA_FLUX_IVAR',
               'HALPHA_BROAD_FLUX','HALPHA_BROAD_FLUX_IVAR','HALPHA_BROAD_VSHIFT','HALPHA_BROAD_SIGMA',
               'NII_6584_FLUX','NII_6584_FLUX_IVAR',
               'SII_6716_FLUX','SII_6716_FLUX_IVAR',
               'SII_6731_FLUX','SII_6731_FLUX_IVAR']

fsf_meta_cols=['TARGETID','SURVEY','PROGRAM','PHOTSYS','LS_ID',
               'FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z','FIBERTOTFLUX_G','FIBERTOTFLUX_R','FIBERTOTFLUX_Z',
               'FLUX_G','FLUX_R','FLUX_Z','FLUX_W1','FLUX_W2','FLUX_W3','FLUX_W4',
               'FLUX_IVAR_G','FLUX_IVAR_R','FLUX_IVAR_Z','FLUX_IVAR_W1','FLUX_IVAR_W2','FLUX_IVAR_W3','FLUX_IVAR_W4',
               'EBV','MW_TRANSMISSION_G','MW_TRANSMISSION_R','MW_TRANSMISSION_Z',
               'MW_TRANSMISSION_W1','MW_TRANSMISSION_W2','MW_TRANSMISSION_W3','MW_TRANSMISSION_W4']

In [ ]:
%%time 
# takes ~ 5min sec
## Directly only read the columns of interest (faster)
fsf_t_cut = Table(fitsio.read(fastspec_file, columns=fsf_data_cols, ext=1))
fsf_m_cut = Table(fitsio.read(fastspec_file, columns=fsf_meta_cols, ext=2))

In [ ]:
%%time
## Really slow with join; is it faster to use hstack? Yes, much faster!
T_fsf_cut = hstack([fsf_m_cut, fsf_t_cut], join_type='inner')
#T_fsf_cut = join(fsf_m_cut, fsf_t_cut, join_type='left', keys=keys_for_join)


In [ ]:
T_fsf_cut.remove_columns(['TARGETID_2','SURVEY_2','PROGRAM_2'])

In [ ]:
T_fsf_cut.rename_columns(['TARGETID_1','SURVEY_1','PROGRAM_1'],\
                         ['TARGETID','SURVEY','PROGRAM'])

In [ ]:
# free memory
del fsf_m_cut
del fsf_t_cut
gc.collect()

In [ ]:
# for DR1, fsf has N=17995820; and with our cuts: 17995653
print(len(T_fsf_cut))
T_fsf_cut[:4]

In [ ]:
## Doesn't seem to help
##%%time
#T_qsom_zpixsum.add_index(['TARGETID'])
#T_fsf_cut.add_index(['TARGETID'])
#T_qsom_zpixsum.add_index(['TARGETID','SURVEY','PROGRAM'])
#T_fsf_cut.add_index(['TARGETID','SURVEY','PROGRAM'])

In [ ]:
%%time
## Takes really long even with index, not set properly?
## SJ: using an INNER join to only keep entries that are also in Fastspecfit
## (for V2.0, keep more QSOs and re-run Fastspecfit where needed)
T = join(T_qsom_zpixsum, T_fsf_cut, join_type='inner', keys=keys_for_join)
#input_table = join(T_qsom_zpixsum, T_fsf_cut, join_type='left', keys=keys_for_join)
print(len(T))

In [ ]:
print(len(T_fsf_cut))

In [ ]:
del T_fsf_cut
del T_qsom_zpixsum
gc.collect()

In [ ]:
gc.collect()

In [ ]:
#print("Difference of 17995653-17995820=", 17995653-17995820)

## 5. Read Yaml

In [ ]:
import importlib
importlib.reload(set_agn_masksDESI)
from set_agn_masksDESI import get_agn_maskbits
from set_agn_masksDESI import update_agn_maskbits

# From Ben (Steph couldn't get this to work)
#from ..py.set_agn_masksDESI import get_agn_maskbits
#from ..py.set_agn_masksDESI import update_agn_maskbits

AGN_MASKBITS, OPT_UV_TYPE, IR_TYPE = get_agn_maskbits('../py/agnmask.yaml')

In [ ]:
AGN_MASKBITS

In [ ]:
OPT_UV_TYPE

In [ ]:
IR_TYPE

## 6. Set QSO_MASKBITS part of AGN_MASKBITS

In [ ]:
%%time 
#takes ~20 sec for DR1
T = update_agn_maskbits(T, AGN_MASKBITS, snr=3, snr_oi=1, snr_oii=1, kewley01=False, mask=None)

In [ ]:
T.columns

In [ ]:
## Sanity check: are there several different values for the new agn_maskbits column?
#print(np.unique(T['AGN_MASKBITS']))

## 7. Set diagnostic bits 

In [ ]:
# From Ben (Steph gets error message)
#from ..py.set_agn_masksDESI import *
from set_agn_masksDESI import *

In [ ]:
%%time
## OPT_UV_TYPE
T = update_agntype_nii_bpt(T, OPT_UV_TYPE, snr=3, mask=None)
T = update_agntype_sii_bpt(T, OPT_UV_TYPE, snr=3, kewley01=False, mask=None)
T = update_agntype_oi_bpt(T, OPT_UV_TYPE, snr=3,snr_oi=1, kewley01=False, mask=None)
T = update_agntype_whan(T, OPT_UV_TYPE, snr=3, mask=None)
T = update_agntype_blue(T, OPT_UV_TYPE, snr=3, snr_oii=3, mask=None)
T = update_agntype_mex(T, OPT_UV_TYPE, snr=3, mask=None)
T = update_agntype_kex(T, OPT_UV_TYPE, snr=3, mask=None)
T = update_agntype_heii(T, OPT_UV_TYPE, snr=3, mask=None)
T = update_agntype_nev(T, OPT_UV_TYPE, snr=2.5, mask=None)
## IR_TYPE
T = update_agntype_wise_colors(T, IR_TYPE, snr=3, mask=None)

In [ ]:
#print(np.unique(T['OPT_UV_TYPE']))

In [ ]:
## Example case to check that some number are set as [NII]-BPT LINERs
is_nii_liner = (T['OPT_UV_TYPE'] & OPT_UV_TYPE.NII_SF !=0)
len(T[is_nii_liner])

In [ ]:
## Example case to check that some number are set
is_kex_agn = (T['OPT_UV_TYPE'] & OPT_UV_TYPE.KEX_AGN !=0)
len(T[is_kex_agn])

In [ ]:
#T.columns

## 8. Join multiwavelength surveys

### NOTE: this should be done OUTSIDE of this workflow and could be run once on all targets (observed and unobserved)

In [ ]:
# sdss, X-ray, IR

## 9. Save catalog

For adding units: https://github.com/desihub/desiutil/blob/main/py/desiutil/annotate.py

However, adding units with annotate_table() does not update the checksum --> need to save file and use annotate_fits() instead.

In [ ]:
## Units for Extension 1
cols_ext1 = ['TARGET_RA', 'TARGET_DEC', 'MEAN_MJD', 'MIN_MJD', 'MAX_MJD', 'COADD_EXPTIME']
units_ext1 = ['deg', 'deg', 'd', 'd', 'd', 's']
units_dict1 = dict(zip(cols_ext1,units_ext1))

units_dict1

In [ ]:
%%time
T_final_cols = T[final_cols]

In [ ]:
%%time
T_sec_ext = T[ext2_cols]

In [ ]:
## SJ: will need to use annotate_fits() instead because this doesn't update the checksum
## Annotate_table only works for extension 1 (will need to re-write the file for ext 2)
#T_final_cols = annotate_table(T_final_cols, units_dict1)

In [ ]:
del T
gc.collect()

In [ ]:
gc.collect()

In [ ]:
%%time

## PROBLEM WITH DR1: Kernel keeps dying

## Write to a temporary path then copy over to an official version number 
## and don't allow to overwrite 
# e.g., /global/cfs/cdirs/desi/science/gqp/agncatalog/edr/v1.0/agnqso_sum.fits

## This will become the official catalog: "agnqso_sum_v1.x.fits" with an updated Version number
primary_hdu = fits.PrimaryHDU()
table1_hdu = fits.BinTableHDU(T_final_cols)
table2_hdu = fits.BinTableHDU(T_sec_ext)
table1_hdu.name = 'AGNCAT'
table2_hdu.name = 'AUXDATA'
hdulist = fits.HDUList([primary_hdu, table1_hdu, table2_hdu])
#hdulist.writeto(dir_for_cat+'agnqso_sum_dr1_tmp.fits', overwrite=True)
#hdulist.writeto(dir_for_cat+'agnqso_sum_dr1_v0.9.fits', overwrite=False)
# EDR
#hdulist.writeto(dir_for_cat+'agnqso_sum_v1.8.fits', overwrite=False)
hdulist.writeto(dir_for_cat+'agnqso_sum_edr_tmp.fits', overwrite=True)

In [ ]:
print('done!')

## 10. Add units to existing catalog (for EDR or DR1)

In [ ]:
## Input file
#infile = dir_for_cat+'agnqso_sum_dr1_v0.9.fits' #DR1
#infile = dir_for_cat+'agnqso_sum_v1.8.fits' #EDR
infile = dir_for_cat+'agnqso_sum_edr_tmp.fits' #EDR

## Test: write a different file in case there are issues
outfile = dir_for_cat+'agnqso_sum_edr_units.fits' #EDR-w-units
#outfile = dir_for_cat+'agnqso_sum_dr1_v1.0.fits' #DR1
#outfile = dir_for_cat+'agnqso_sum_v1.9.fits' #EDR

In [ ]:
## Extension 2
cols_readme = 'cols_units_ext2.txt'
tcols_ext2 = Table.read(cols_readme, format='ascii.basic', delimiter='|')
tcols_ext2.remove_columns(['col0', 'Format', '_1'])

## Convert to dictionary to use annotate_fits
cols_ext2 = list(tcols_ext2['Name'].data)
units_ext2 = list(tcols_ext2['Units'].data)
units_dict2 = dict(zip(cols_ext2,units_ext2))

units_dict2

In [ ]:
%%time
from desiutil.annotate import annotate_fits
annotate_fits(infile, 2, outfile, units=units_dict2, overwrite=True, validate=False)

In [ ]:
## Units for Extension 1
cols_ext1 = ['TARGET_RA', 'TARGET_DEC', 'MEAN_MJD', 'MIN_MJD', 'MAX_MJD', 'COADD_EXPTIME']
units_ext1 = ['deg', 'deg', 'd', 'd', 'd', 's']
units_dict1 = dict(zip(cols_ext1,units_ext1))

units_dict1

In [ ]:
%%time
# Overwrite the file in the case of adding units to ext[1]
annotate_fits(outfile, 1, outfile, units=units_dict1, overwrite=True, validate=True)

## 11. Fix for DR1 to remove LS_ID=999999

In [ ]:
print('test')

In [ ]:
%%time
## Input/output file
#infile = dir_for_cat+'agnqso_sum_dr1_v1.0.fits'
#outfile = dir_for_cat+'agnqso_sum_dr1_v1.0_fix.fits'
infile = dir_for_cat+'agnqso_sum_edr_units.fits' #EDR-w-units
outfile = dir_for_cat+'agnqso_sum_edr_fix.fits'

t = Table(fitsio.read(infile, ext=1))
t2 = Table(fitsio.read(infile, ext=2))

In [ ]:
is_null = t['LS_ID']==999999

In [ ]:
print(len(t[is_null]))

In [ ]:
t = t[~is_null]
t2 = t2[~is_null]

In [ ]:
%%time

## This will become the official catalog: "agnqso_sum_v1.x.fits" with an updated Version number
primary_hdu = fits.PrimaryHDU()
table1_hdu = fits.BinTableHDU(t)
table2_hdu = fits.BinTableHDU(t2)
table1_hdu.name = 'AGNCAT'
table2_hdu.name = 'AUXDATA'
hdulist = fits.HDUList([primary_hdu, table1_hdu, table2_hdu])
hdulist.writeto(outfile, overwrite=True)

In [ ]:
## Might need to restart kernel and clear outputs before running this:
## Extension 2
cols_readme = 'cols_units_ext2.txt'
tcols_ext2 = Table.read(cols_readme, format='ascii.basic', delimiter='|')
tcols_ext2.remove_columns(['col0', 'Format', '_1'])

## Convert to dictionary to use annotate_fits
cols_ext2 = list(tcols_ext2['Name'].data)
units_ext2 = list(tcols_ext2['Units'].data)
units_dict2 = dict(zip(cols_ext2,units_ext2))

units_dict2

In [ ]:
%%time
# Overwrite the temporary outfile with _fix in the name
outfile = dir_for_cat+'agnqso_sum_dr1_v1.0_fix.fits'

annotate_fits(outfile, 2, outfile, units=units_dict2, overwrite=True, validate=False)

## Sanity check on LS_ID

In [ ]:
%%time
#file = dir_for_cat+'agnqso_sum_dr1_v1.0_fix.fits' #DR1
file = dir_for_cat+'agnqso_sum_dr1_v1.0.fits' #DR1

t = Table(fitsio.read(file, columns=['TARGETID','LS_ID','Z','SURVEY','DESI_TARGET','SCND_TARGET'], ext=1))

In [ ]:
print(len(t))
t[:3]

In [ ]:
is_null = t['LS_ID']==999999
is_zero = t['LS_ID']==0

N = len(t)
N_null = len(t[is_null])
N_zero = len(t[is_zero])

print('Null', N_null, 'Fraction:',np.round(N_null/N, 3))
print('Zero', N_zero, 'Fraction:',np.round(N_zero/N, 3))

In [ ]:
print(len(t[is_null&(t['SURVEY']!='main')]))
print(len(t[is_null&(t['SURVEY']=='main')]))

In [ ]:
#print(np.min(t['TARGETID'][is_null]), np.max(t['TARGETID'][is_null]))
print(np.min(t['TARGETID'][is_zero]), np.max(t['TARGETID'][is_zero]))
print(np.min(t['TARGETID'][~is_zero&~is_null]), np.max(t['TARGETID'][~is_zero&~is_null]))

In [ ]:
from desitarget.targets import decode_targetid

def get_release(targetid_in):
    objid, brickid, release, _, _, _ = decode_targetid(targetid_in)
    return(release)

def convert_targetid_lsid(targetid_in):
    objid, brickid, release, _, _, _ = decode_targetid(targetid_in)
    ls_id_out = (release<<40)|(brickid<<16)|(objid)
    return(ls_id_out)


In [ ]:
tid = 103606506225676

decode_targetid(tid)

In [ ]:
t_null = t[is_null] #&(t['SURVEY']=='main')]
#t_null = t[is_null&(t['SURVEY']=='main')]
N_test = len(t_null)
t_null['RELEASE'] = -99

In [ ]:

for i in range(N_test):
    t_null['RELEASE'][i] = get_release(t_null['TARGETID'][i])
    

In [ ]:
print(np.unique(t_null['RELEASE']))

In [ ]:
releases = [9010, 9011, 9012, 9999]

print('N(zero)', len(t_null))

for rel in releases:
    print("RELEASE=",rel, 'N=',len(t_null[t_null['RELEASE']==rel]))
#    print("... of which, SCDN:", len(t_null[(t_null['RELEASE']==rel)&(t_null['SCND_TARGET']>0)]))

print("RELEASE from 0 to 8888; N=",len(t_null[t_null['RELEASE']<=8888]))

In [ ]:
t_null[t_null['RELEASE']==9011]

In [ ]:
#prefix = 'https://www.legacysurvey.org/viewer-data/desi-spectrum/dr1/targetid'
prefix = 'https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid'

for i in range(3):
    print(prefix+str(t_null['TARGETID'][i]))

## Below, only for EDR

In [ ]:
## Input file
infile = dir_for_cat+'agnqso_sum_v1.8.fits' #EDR

## Test: write a different file in case there are issues
outfile = dir_for_cat+'agnqso_sum_v1.9.fits' #EDR

In [ ]:
%%time
## Catalog HDU List
agn_hdul = fits.open(infile, format='fits')

# Load the catalog into Astropy tables
T = Table(agn_hdul[1].data)
T2 = Table(agn_hdul[2].data)

In [ ]:
## Extension 2
cols_readme = 'cols_units_ext2.txt'
tcols_ext2 = Table.read(cols_readme, format='ascii.basic', delimiter='|')
tcols_ext2.remove_columns(['col0', 'Format', '_1'])
tcols_ext2[:3]

In [ ]:
T2_cols = Table()
T2_cols['Name'] = T2.colnames
print(len(T2_cols))
T2_cols[:3]

In [ ]:
T2_cols_info = join(T2_cols, tcols_ext2)

In [ ]:
print(len(T2_cols_info))
T2_cols_info

In [ ]:
## Convert to dictionary to use annotate_table
cols_ext2 = list(T2_cols_info['Name'].data)
units_ext2 = list(T2_cols_info['Units'].data)
units_dict2 = dict(zip(cols_ext2,units_ext2))

#units_dict2

In [ ]:
from desiutil.annotate import annotate_table

T2 = annotate_table(T2, units_dict2, validate=True)

In [ ]:
%%time
T2.write('test.fits',overwrite=True)

In [ ]:
T2

In [ ]:
T = annotate_table(T, units_dict1)

In [ ]:
primary_hdu = fits.PrimaryHDU()
table1_hdu = fits.BinTableHDU(T)
table2_hdu = fits.BinTableHDU(T2)
table1_hdu.name = 'AGNCAT'
table2_hdu.name = 'AUXDATA'
hdulist = fits.HDUList([primary_hdu, table1_hdu, table2_hdu])
hdulist.writeto(outfile, overwrite=False)

## Check the file

In [ ]:
file = dir_for_cat+'agnqso_sum_dr1_v1.0.fits' #DR1
hdul = fits.open(file)

In [ ]:
hdul[1].header

In [ ]:
hdul[2].header

In [ ]:
outfile = dir_for_cat+'agnqso_sum_v1.9.fits'

#annotate_fits(outfile, 2, outfile, units=units_dict2, overwrite=True)
